# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * LangChain Expression Language (LCEL)
  * Sequential Chains with LCEL
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [4]:
import pandas as pd
df = pd.read_csv('data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [7]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [8]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [9]:
chain = prompt | llm

In [10]:
product = "Queen Size Sheet Set"
chain.invoke({"product": product})

AIMessage(content='"Royal Bedding Co."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 24, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C5ZZjBFQRyy360DeuLIXnmGlDqv8j', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d1e40336-02e7-40c7-8935-587cc52b850a-0', usage_metadata={'input_tokens': 24, 'output_tokens': 6, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## LangChain Expression Language (LCEL)

In [11]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = first_prompt | llm

In [12]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = second_prompt | llm

In [13]:
overall_simple_chain = (
    {"company_name": first_prompt | llm} | second_prompt | llm
)

In [14]:
response = overall_simple_chain.invoke({"product": "t-shirts"})

print(response.content)

TeeTopia offers trendy and comfortable t-shirts for all ages, featuring unique designs and high-quality materials. Upgrade your wardrobe today!


## Sequential Chains with LCEL

In [15]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [16]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = {"English_Review": first_prompt | llm}

In [17]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = {"summary": second_prompt | llm}


In [18]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = {"language": third_prompt | llm}

In [19]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = {"followup_message": fourth_prompt | llm}

In [20]:
parallel_inputs = RunnableParallel(
    English_Review=chain_one,
    language=chain_three,
    Review=RunnablePassthrough()
)

overall_chain = (
    RunnablePassthrough.assign(
        intermediate_output=parallel_inputs
    )
    | {
        "summary": lambda x: (second_prompt | llm).invoke({"English_Review": x["intermediate_output"]["English_Review"]}),
        "language": lambda x: x["intermediate_output"]["language"],
        "English_Review": lambda x: x["intermediate_output"]["English_Review"],
    }
    | fourth_prompt | llm
)

In [21]:
review = df.Review[5]
response = overall_chain.invoke({"Review": review})

print(response)

content="Je suis désolé d'apprendre que votre expérience avec le produit n'a pas été satisfaisante. Il est important de vérifier l'authenticité du produit pour éviter les contrefaçons. Il peut être utile de contacter le fabricant pour obtenir des clarifications sur la qualité du lot que vous avez reçu. Nous apprécions vos commentaires et nous ferons de notre mieux pour améliorer votre expérience à l'avenir. Merci de nous avoir informés de cette question." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 580, 'total_tokens': 692, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C5ZZn681rEUICqdimPFXYUvDaYT5Q', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--9c7

## Router Chain

In [22]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [23]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [24]:
from langchain_core.runnables import RunnableBranch
from langchain.chains.router.llm_router import RouterOutputParser
from langchain.prompts import PromptTemplate

In [25]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [26]:
destination_chains = {
    info["name"]: PromptTemplate.from_template(info["prompt_template"]) | llm
    for info in prompt_infos
}

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [27]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = default_prompt | llm

In [28]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a language model select the model prompt best suited for the input.
You will be given the names of the available prompts and a description of what the prompt is best suited for.
You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string # "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string # a potentially modified version of the original input
}}
```

REMEMBER: The value of “destination” MUST match one of the candidate prompts listed below.
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{input}

<< OUTPUT (remember to include the ```json)>>"""

In [29]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.replace(
    "{destinations}", destinations_str
    )

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = (
    router_prompt
    | llm
    | RouterOutputParser()
)

In [30]:
routing_branch = RunnableBranch(
    (
        lambda x: x["router_output"]["destination"] in destination_chains,
        lambda x: destination_chains[x["router_output"]["destination"]].invoke(
            {"input": x["input"]}
        ),
    ),
    default_chain,
)

overall_chain = RunnableParallel(
    # Get the router's output and store it under the 'router_output' key
    router_output=router_chain,
    # Pass the output of the router and the original input to the routing branch
    final_answer=RunnablePassthrough.assign(
        router_output=router_chain
    ) | routing_branch,
)

response = overall_chain.invoke({"input": "What is black body radiation?"})
print(f"Router chose: {response['router_output']['destination']}")
print(f"Final Answer: {response['final_answer'].content}")

Router chose: physics
Final Answer: Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics.


In [31]:
response = overall_chain.invoke({"input": "what is 2 + 2"})
print(f"Router chose: {response['router_output']['destination']}")
print(f"Final Answer: {response['final_answer'].content}")

Router chose: math
Final Answer: The answer to 2 + 2 is 4.


In [32]:
response = overall_chain.invoke({"input": "Why does every cell in our body contain DNA?"})
print(f"Router chose: {response['router_output']['destination']}")
print(f"Final Answer: {response['final_answer'].content}")

Router chose: None
Final Answer: Every cell in our body contains DNA because DNA carries the genetic information that determines the characteristics and functions of an organism. DNA contains the instructions for building and maintaining an organism, including the proteins that are essential for cell function and structure. This genetic information is passed down from parent to offspring and is essential for the growth, development, and functioning of all cells in the body. Having DNA in every cell ensures that the genetic information is preserved and can be used to carry out the necessary processes for life.
